In [26]:
import kagglehub
import os
import pandas as pd

# Download latest version
folder = kagglehub.dataset_download("rohitsahoo/sales-forecasting")
path = folder + "/train.csv"

df = pd.read_csv(path)

In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",         
    user="new_user",     
    password="user", 
    database="db",  
    port=3311
)

cursor = connection.cursor()

In [ ]:
df.isnull().sum()

In [27]:
df = df.where(pd.notna(df), None)

In [28]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df['Postal Code'] = df['Postal Code'].fillna(0).astype(int)

In [29]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])

In [30]:
df.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
dtype: object

In [16]:
for index, row in df.iterrows():
    sql = """
    INSERT INTO orders (row_id, order_id, order_date, ship_date, ship_mode, customer_id, 
                        customer_name, segment, country, city, state, postal_code, region, 
                        product_id, category, sub_category, product_name, sales)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (
        row['Row ID'], row['Order ID'], row['Order Date'], row['Ship Date'], row['Ship Mode'], 
        row['Customer ID'], row['Customer Name'], row['Segment'], row['Country'], row['City'], 
        row['State'], row['Postal Code'], row['Region'], row['Product ID'], row['Category'], 
        row['Sub-Category'], row['Product Name'], row['Sales']
    ))


connection.commit()
cursor.close()
connection.close()

Code to fill in the data for the dimension tables

In [17]:
df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94


In [24]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",         
    user="new_user",     
    password="user", 
    database="db",  
    port=3311
)

cursor = connection.cursor()

In [31]:
query_location = """insert into location (postal_code, country, region, city, state)
SELECT * FROM (SELECT %s, %s, %s, %s, %s) AS tmp
WHERE NOT EXISTS (
    SELECT 1 FROM location WHERE postal_code = %s
);
"""

query_customer = """insert into customer (customer_id, customer_name, segment)
SELECT * FROM (SELECT %s, %s, %s) AS tmp
WHERE NOT EXISTS (
    SELECT 1 FROM customer WHERE customer_id = %s
);
"""

query_product = """insert into product (product_id , category, sub_category, product_name)
SELECT * FROM (SELECT %s, %s, %s, %s) AS tmp
WHERE NOT EXISTS (
    SELECT 1 FROM product WHERE product_id = %s
);
"""

query_order = """insert into order_dim (order_id , order_date, ship_date, ship_mode)
SELECT * FROM (SELECT %s, %s, %s, %s) AS tmp
WHERE NOT EXISTS (
    SELECT 1 FROM order_dim WHERE order_id = %s
);
"""

query_sales = """insert into sales (row_id, postal_code, customer_id, product_id, order_id, sales)
values (%s, %s, %s, %s, %s, %s) 
);
"""

In [ ]:
batch_size = 0

for _, row in df.iterrows():
    if batch_size > 5:
        break
    cursor.execute(query_location, (
        row['Postal Code'], row['Country'], row['Region'], row['City'], row['State'], row['Postal Code']
    ))

    # if batch_size < 2:
    #     print("after location")

    # cursor.execute(query_customer, (
    #     row['Customer ID'], row['Customer Name'], row['Segment'], row['Customer ID']
    # ))

    # if batch_size < 2:
    #     print("after customer")

    # cursor.execute(query_product, (
    #     row['Product ID'], row['Category'], row['Sub-Category'], row['Product Name'], row['Product ID']
    # ))

    # if batch_size < 2:
    #     print("after product")

    # cursor.execute(query_order, (
    #     row['Order ID'], row['Order Date'], row['Ship Date'], row['Ship Mode'], row['Order ID']
    # ))

    # if batch_size < 2:
    #     print("after order")

    # cursor.execute(query_sales, (
    #     row['Row ID'], row["Postal Code"], row['Customer ID'], row['Product ID'], row['Order ID'], row['Sales']
    # ))

    # if batch_size < 2:
    #     print("after sales")

    # batch_size += 1

    # if batch_size % 100 == 0:
    #     connection.commit()

connection.commit()
cursor.close()
connection.close()

DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction